## Anna Jazayeri 131661209
### Workshop 5
### Text Mining BDM 550

In [1]:
import zipfile
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
import re

from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import *
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from nltk.corpus import PlaintextCorpusReader

## <span style="color:blue;">Task 1: Preprocess the text to remove any stop words or punctuations.</span>

In [2]:
# first of all I must loading the files 
# import zipfile
# Download and unzip the labeled and unlabeled datasets
with zipfile.ZipFile('labeled_dataset.zip', 'r') as labeled_zip:
    labeled_zip.extractall('labeled_dataset')

with zipfile.ZipFile('unlabeled_dataset.zip', 'r') as unlabeled_zip:
    unlabeled_zip.extractall('unlabeled_dataset')

In [3]:
# assigning the labeled and unlabeled datasets
label = 'labeled_dataset'
unlabel = 'unlabeled_dataset'

In [4]:
# Now I must remove  the stopwords and punctuations
# I create a funcion for this 
# from nltk.corpus import stopwords
# import os
# from nltk.tokenize import word_tokenize
# import nltk

def remove_stop_punc(txt):
    # Tokenizing the text
    tokens = nltk.word_tokenize(txt)
    # removing the stopwords and punctuations
    tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stopwords.words('english')]
    # now the cleaned tokens must be join together as a string
    clean_txt = ' '.join(tokens)
    return clean_txt

# now we should use a for loop to cleaning each corpura and then 
# and then storing those cleaned corpura inside of a list 
clean_label = []
clean_unlabel = []

# a for loop for cleaning the labeled texts
for file in os.listdir(label):
    if os.path.isfile(os.path.join(label, file)):
        with open(os.path.join(label, file), 'r', encoding='utf-8') as file:
            for line in file:
                clean_label.append(remove_stop_punc(line.strip()))

                
# a for loop for cleaning the unlabeled texts
for file in os.listdir(unlabel):
    if os.path.isfile(os.path.join(unlabel, file)):
        with open(os.path.join(unlabel, file), 'r', encoding='utf-8') as file:
            for line in file:
                clean_unlabel.append(remove_stop_punc(line.strip()))
                

In [5]:
# lets see our clean label list just for first 5 lines
clean_label[:5]

['abstract',
 'misc although internet level topology extensively studied past years little known details taxonomy',
 'misc node represent wide variety organizations e g large isp small private business university vastly different network characteristics external connectivity patterns network growth tendencies properties hardly neglect working veracious internet representations simulation environments',
 'aimx paper introduce radically new approach based machine learning techniques map ases internet natural taxonomy',
 'ownx successfully classify number number percent ases expected accuracy number number percent']

In [6]:
# lets see our clean unlabel list just for first 5 lines
clean_unlabel[:5]

['abstract',
 'transporter analyses conducted 141 organisms whose complete genome sequences available',
 'organism complete set membrane transport systems identified predicted functions classified protein families based transporter classification system',
 'organisms larger genome sizes generally possessed relatively greater number transport systems',
 'prokaryotes unicellular eukaryotes significant factor increase transporter content genome size greater diversity transporter types']

## <span style="color:blue;">Task 2: Use TF-IDF to vectorize the sentences.</span>

In [7]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizing with TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()
# Fiting and transforming our clean label data
tfidf_label = tfidf_vectorizer.fit_transform(clean_label)
# Fiting and transforming our clean unlabel data
tfidf_unlabel = tfidf_vectorizer.transform(clean_unlabel)

## <span style="color:blue;">Task 3: Use Scikit learn to create a (naïve bayes) classifier.</span>

<span style="color:blue;">**First create a classifier (naïve bayes) to classify the given dataset into 6 categories: (AIMX, OWNX, CONT, BASE, NUMBER, and MISC).<br>Then, use the classifier to label the sentences in the unlabeled dataset.**</span>



In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have the labels for the labeled dataset
# You can replace this with the actual labels you have
labels = ["AIMX", "OWNX", "CONT", "BASE", "NUMBER", "MISC"]

# Creating a Naive Bayes classifier
classifier = make_pipeline(TfidfTransformer(), MultinomialNB())

# Splitting the labeled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_label, labels, test_size=0.2, random_state=42)

# Training the classifier
classifier.fit(X_train, y_train)

# Predicting labels for the test set
y_pred = classifier.predict(X_test)

# Evaluating the classifier
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Now, use the trained classifier to predict labels for the unlabeled dataset
predicted_labels = classifier.predict(tfidf_unlabel)

# Print the predicted labels for the unlabeled dataset
for i, label in enumerate(predicted_labels):
    print(f"Document {i + 1}: Predicted Label - {label}")


ValueError: Found input variables with inconsistent numbers of samples: [335, 6]

## <span style="color:blue;">Task 4: Summarize your work and your findings in a few sentences.</span>

### <span style="color:blue;">BONUS [Extra 5% added to any workshops]: Use other classification approaches to label the unlabeled sentences. Evaluate your work using precision, recall, and f1 score.</span>